In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torch.autograd as autograd
import numpy as np

import argparse
from collections import defaultdict
from copy import copy
from distutils.command.config import config
from foo import Nop
import numpy as np
import torch
import random
import os

device = torch.device("cpu")

#fix seed
seed=42
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Python RNG
np.random.seed(seed)
random.seed(seed)

In [10]:
def stack_observations(env):
    obs = []
    for s in range(env.num_states):
        obs.append(env.observation(s))
    return np.stack(obs)
    
class FCNetwork(torch.nn.Module):
    def __init__(self, env, layers=[20,20]):
        super(FCNetwork, self).__init__()
        self.all_observations = torch.tensor(stack_observations(env), dtype=torch.float32).to(device)
        dim_input = env.dim_obs
        dim_output = env.num_actions
        net_layers = []

        dim = dim_input
        for i, layer_size in enumerate(layers):
            net_layers.append(torch.nn.Linear(dim, layer_size))
            net_layers.append(torch.nn.ReLU())
            dim = layer_size
        net_layers.append(torch.nn.Linear(dim, dim_output))
        self.layers = net_layers
        self.network = torch.nn.Sequential(*net_layers).to(device)

    def forward(self, states):
        observations = torch.index_select(self.all_observations.to(device), 0, states.to(device))
        return self.network(observations)

In [11]:
path = '/home/asap7772/asap7772/algo_gridworld/data/exp_maze5_datasetmixed_limited_skewed_size-2_envsmooth_minq0.1_ttype13_const2.0/net_limited.pt'
net = torch.load(path)

In [12]:
net

FCNetwork(
  (network): Sequential(
    (0): Linear(in_features=8, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=5, bias=True)
  )
)